# 3D FEA — CalculiX (Full Spring, Extruded)

Full 3D CalculiX analysis of the **complete doubly-clamped bistable spring**
with `C3D10` (10-node quadratic tetrahedral) elements.

**Boundary conditions**:
- Left anchor (x≈0): clamped, u_x = u_y = u_z = 0
- Right anchor (x≈anchor_distance): clamped, u_x = u_y = u_z = 0
- Shuttle (center): prescribe u_y, u_x and u_z free

Runs at 3 mesh densities, then includes **cross-method comparison** of all
full-spring results (4a/4b/5a/5b + analytical).

In [ ]:
import sys
import os
import subprocess
import csv
import numpy as np
import matplotlib.pyplot as plt
import shutil

sys.path.insert(0, os.path.dirname(os.path.abspath("__file__")))
from full_spring_utils import (
    POLY_SI, DEFAULT_FULL_SPRING_PARAMS,
    get_full_spring_polygon, get_full_spring_3d_mesh_order2,
    identify_bc_nodes_3d,
)

P = DEFAULT_FULL_SPRING_PARAMS
anchor_dist = P['anchor_distance']
half_span = P['half_span']
shuttle_len = P['shuttle_length']
h = P['initial_offset']
t = POLY_SI['t']
E = POLY_SI['E']
nu = POLY_SI['nu']

CCX = shutil.which('ccx') or os.path.abspath('../../switch-env/bin/ccx')
print(f"CalculiX: {CCX}")
print(f"Full spring: anchor_dist={anchor_dist} µm, half_span={half_span} µm")
print(f"  h={h} µm, t={t} µm")

os.makedirs('plots', exist_ok=True)
os.makedirs('results', exist_ok=True)

## 1. CalculiX Input Writer

In [ ]:
def write_nset(f, name, nids, per_line=16):
    """Write a node set to CalculiX .inp file."""
    f.write(f'*NSET, NSET={name}\n')
    for j, nid in enumerate(nids):
        f.write(f'{nid}')
        if (j + 1) % per_line == 0 or j == len(nids) - 1:
            f.write('\n')
        else:
            f.write(', ')


def write_ccx_3d_full(filepath, points, elems, elem_type,
                       left_nids, right_nids, shuttle_nids, delta, E, nu):
    """Write CalculiX .inp for 3D full spring.
    
    BCs: left + right anchors clamped (all DOFs=0), shuttle y-prescribed (DOF 2).
    """
    with open(filepath, 'w') as f:
        f.write('*HEADING\n')
        f.write('CCS Full Bistable Spring - 3D\n**\n')
        
        f.write('*NODE\n')
        for i, (x, y, z) in enumerate(points):
            f.write(f'{i+1}, {x:.8f}, {y:.8f}, {z:.8f}\n')
        
        f.write(f'*ELEMENT, TYPE={elem_type}, ELSET=SPRING\n')
        for i, conn in enumerate(elems):
            node_str = ', '.join(str(n+1) for n in conn)
            f.write(f'{i+1}, {node_str}\n')
        
        write_nset(f, 'LEFT_ANCHOR', left_nids)
        write_nset(f, 'RIGHT_ANCHOR', right_nids)
        write_nset(f, 'SHUTTLE', shuttle_nids)
        
        f.write('*MATERIAL, NAME=POLYSI\n')
        f.write('*ELASTIC\n')
        f.write(f'{E:.1f}, {nu:.4f}\n')
        
        f.write('*SOLID SECTION, ELSET=SPRING, MATERIAL=POLYSI\n\n')
        
        f.write('*STEP, NLGEOM, INC=1000\n')
        f.write('*STATIC\n')
        f.write('0.05, 1.0, 1e-8, 0.2\n')
        
        # Left anchor: clamp all 3 DOFs
        f.write('*BOUNDARY\n')
        f.write('LEFT_ANCHOR, 1, 3, 0.0\n')
        
        # Right anchor: clamp all 3 DOFs
        f.write('*BOUNDARY\n')
        f.write('RIGHT_ANCHOR, 1, 3, 0.0\n')
        
        # Shuttle: prescribe y only (DOF 2)
        f.write('*BOUNDARY\n')
        f.write(f'SHUTTLE, 2, 2, {delta:.8f}\n')
        
        f.write('*NODE FILE\n')
        f.write('U, RF\n')
        f.write('*NODE PRINT, NSET=SHUTTLE, TOTALS=YES\n')
        f.write('RF\n')
        
        f.write('*END STEP\n')


def parse_ccx_reaction_force(dat_file):
    """Parse total reaction force (y-component) from .dat file."""
    with open(dat_file, 'r') as f:
        lines = f.readlines()
    rf_y = None
    for i, line in enumerate(lines):
        if 'total force' in line.lower():
            for j in range(i + 1, min(i + 5, len(lines))):
                parts = lines[j].split()
                if len(parts) >= 2:
                    try:
                        rf_y = float(parts[1])
                        break
                    except ValueError:
                        continue
    return rf_y

print("Input writer and parser ready.")

## 2. 3D Mesh Generation + Displacement Sweep (3 densities)

In [ ]:
poly = get_full_spring_polygon(n_points=200)

densities = {
    '10pct': {'lc_flex': 0.8, 'lc_rigid': 1.2, 'n_layers_z': 2,
              'n_steps': 15, 'label': '10% density'},
    '50pct': {'lc_flex': 0.5, 'lc_rigid': 0.8, 'n_layers_z': 2,
              'n_steps': 18, 'label': '50% density'},
    '100pct': {'lc_flex': 0.4, 'lc_rigid': 0.7, 'n_layers_z': 3,
               'n_steps': 20, 'label': '100% density'},
}

all_results = {}

for density_name, cfg in densities.items():
    print(f"\n{'='*60}")
    print(f"Density: {cfg['label']}")
    print(f"  lc_flex={cfg['lc_flex']}, lc_rigid={cfg['lc_rigid']}, n_z={cfg['n_layers_z']}")
    print(f"{'='*60}")
    
    ccx_dir = os.path.abspath(f'results/ccx_3d_full_{density_name}')
    os.makedirs(ccx_dir, exist_ok=True)
    
    # --- 3D mesh (2nd order for C3D10) ---
    points_3d, tets, elem_type = get_full_spring_3d_mesh_order2(
        polygon=poly,
        thickness=t,
        lc_flex=cfg['lc_flex'],
        lc_rigid=cfg['lc_rigid'],
        n_layers_z=cfg['n_layers_z'],
        anchor_distance=anchor_dist,
        half_span=half_span,
        flex_ratio=P['flex_ratio'],
    )
    print(f"Mesh: {len(points_3d)} nodes, {len(tets)} {elem_type} elements")
    
    # --- Boundary nodes (1-indexed) ---
    bc = identify_bc_nodes_3d(points_3d, anchor_dist, half_span, shuttle_len)
    left_nids = bc['left_anchor'] + 1
    right_nids = bc['right_anchor'] + 1
    shuttle_nids = bc['shuttle'] + 1
    print(f"Left: {len(left_nids)}, Right: {len(right_nids)}, Shuttle: {len(shuttle_nids)}")
    
    # --- Sweep ---
    n_steps = cfg['n_steps']
    delta_vals = np.linspace(0, -2*h, n_steps)
    delta_vals[0] = -0.01
    F_ccx = np.zeros(n_steps)
    
    for i, delta in enumerate(delta_vals):
        job = f'step_{i:03d}'
        inp_file = os.path.join(ccx_dir, f'{job}.inp')
        
        write_ccx_3d_full(inp_file, points_3d, tets, elem_type,
                          left_nids, right_nids, shuttle_nids, delta, E, nu)
        
        result = subprocess.run(
            [CCX, '-i', job],
            cwd=ccx_dir,
            capture_output=True, text=True, timeout=300
        )
        
        dat_file = os.path.join(ccx_dir, f'{job}.dat')
        if os.path.exists(dat_file):
            rf = parse_ccx_reaction_force(dat_file)
            if rf is not None:
                F_ccx[i] = rf
            else:
                print(f"  Step {i}: parse failed")
        else:
            print(f"  Step {i}: ccx failed")
        
        if i % 5 == 0:
            print(f"  Step {i}/{n_steps}: δ={delta:.3f} µm, F={F_ccx[i]:.4f} µN")
    
    delta_vals[0] = 0.0
    F_ccx[0] = 0.0
    
    print(f"Done. F range: [{F_ccx.min():.4f}, {F_ccx.max():.4f}] µN")
    
    all_results[density_name] = {
        'delta': -delta_vals,
        'F': F_ccx,
        'n_nodes': len(points_3d),
        'n_elems': len(tets),
    }
    
    np.savetxt(f'results/calculix_3d_full_{density_name}.csv',
               np.column_stack([-delta_vals, F_ccx]),
               delimiter=',', header='delta_um,F_uN', comments='')
    print(f"Saved: results/calculix_3d_full_{density_name}.csv")

## 3. Mesh Convergence Plot

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors = {'10pct': 'green', '50pct': 'blue', '100pct': 'red'}
for name, res in all_results.items():
    label = f"{densities[name]['label']} ({res['n_nodes']} nodes, {res['n_elems']} elems)"
    ax.plot(res['delta'], res['F'], 's-', color=colors[name], linewidth=1.5,
            ms=2, label=label)

try:
    ana = np.loadtxt('results/analytical_force_displacement.csv',
                     delimiter=',', skiprows=1)
    ax.plot(ana[:, 0], 4 * ana[:, 2], 'k:', linewidth=1.5,
            label='Analytical CCS (×4)')
except FileNotFoundError:
    pass

ax.axhline(0, color='gray', linewidth=0.5)
ax.set_xlabel('Shuttle displacement from initial [µm]')
ax.set_ylabel('Force F [µN]')
ax.set_title('CalculiX 3D Full Spring — Mesh Convergence')
ax.legend()
ax.grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig('plots/calculix_3d_full_convergence.png', dpi=150, bbox_inches='tight')
plt.show()

---
# Cross-Method Comparison — Full Spring

Overlay all full-spring force-displacement curves from notebooks 4a, 4b, 5a, 5b
plus the analytical model (scaled ×4 for 4 half-beams).

In [ ]:
# Load all available full-spring results
comparison = {}

file_specs = [
    ('Analytical CCS (×4)', 'results/analytical_force_displacement.csv', 2, 4.0),
    ('scikit-fem 2D (100%)', 'results/skfem_2d_full_100pct.csv', 1, 1.0),
    ('scikit-fem 2D (50%)', 'results/skfem_2d_full_50pct.csv', 1, 1.0),
    ('scikit-fem 2D (10%)', 'results/skfem_2d_full_10pct.csv', 1, 1.0),
    ('scikit-fem 3D (100%)', 'results/skfem_3d_full_100pct.csv', 1, 1.0),
    ('scikit-fem 3D (50%)', 'results/skfem_3d_full_50pct.csv', 1, 1.0),
    ('scikit-fem 3D (10%)', 'results/skfem_3d_full_10pct.csv', 1, 1.0),
    ('CalculiX 2D (100%)', 'results/calculix_2d_full_100pct.csv', 1, 1.0),
    ('CalculiX 2D (50%)', 'results/calculix_2d_full_50pct.csv', 1, 1.0),
    ('CalculiX 2D (10%)', 'results/calculix_2d_full_10pct.csv', 1, 1.0),
    ('CalculiX 3D (100%)', 'results/calculix_3d_full_100pct.csv', 1, 1.0),
    ('CalculiX 3D (50%)', 'results/calculix_3d_full_50pct.csv', 1, 1.0),
    ('CalculiX 3D (10%)', 'results/calculix_3d_full_10pct.csv', 1, 1.0),
]

for name, fpath, col, scale in file_specs:
    try:
        data = np.loadtxt(fpath, delimiter=',', skiprows=1)
        comparison[name] = {'delta': data[:, 0], 'F': scale * data[:, col]}
        print(f"Loaded: {name} ({len(data)} points)")
    except FileNotFoundError:
        print(f"Not found: {fpath}")

In [ ]:
# Comparison plot — finest mesh from each method
fig, ax = plt.subplots(figsize=(12, 7))

# Plot only 100% (finest) from each method + analytical
plot_specs = [
    ('Analytical CCS (×4)',  'black', '-',  2.0, ''),
    ('scikit-fem 2D (100%)', 'blue',  '-',  1.5, ''),
    ('scikit-fem 3D (100%)', 'blue',  '--', 1.5, ''),
    ('CalculiX 2D (100%)',   'red',   '-',  1.5, 'o'),
    ('CalculiX 3D (100%)',   'red',   '--', 1.5, 's'),
]

for name, color, ls, lw, marker in plot_specs:
    if name in comparison:
        d = comparison[name]
        ax.plot(d['delta'], d['F'], color=color, linestyle=ls, linewidth=lw,
                marker=marker, markersize=2, label=name)

ax.axhline(0, color='gray', linewidth=0.5)
ax.axvline(h, color='gray', linewidth=0.5, linestyle=':')
ax.set_xlabel('Shuttle displacement from initial [µm]', fontsize=12)
ax.set_ylabel('Force F [µN]', fontsize=12)
ax.set_title('Full CCS Bistable Spring — Force-Displacement Comparison', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig('plots/full_spring_comparison.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Critical values table
print(f"{'Method':30s} {'F_push [µN]':>12s} {'F_pop [µN]':>12s} {'|F_pop/F_push|':>14s}")
print('-' * 70)

rows = []
for name, d in comparison.items():
    if '10%' in name or '50%' in name:
        continue  # only show finest
    F = d['F']
    delta = d['delta']
    F_push = F.max()
    F_pop = F.min()
    ratio = abs(F_pop / F_push) if abs(F_push) > 1e-10 else 0.0
    
    idx_peak = np.argmax(F)
    zero_after = np.where(np.diff(np.sign(F[idx_peak:])))[0]
    delta_snap = delta[idx_peak + zero_after[0]] if len(zero_after) > 0 else 0.0
    
    print(f"{name:30s} {F_push:12.4f} {F_pop:12.4f} {ratio:14.4f}")
    rows.append([name, f'{F_push:.6f}', f'{F_pop:.6f}', f'{ratio:.4f}',
                 f'{delta_snap:.4f}', f'{2*h:.4f}'])

# Save
with open('results/full_spring_comparison.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['method', 'F_push_uN', 'F_pop_uN', 'push_pop_ratio',
                     'delta_snap_um', 'total_travel_um'])
    writer.writerows(rows)

print(f"\nSaved: results/full_spring_comparison.csv")
print(f"\nExpected: F_push ≈ 10 µN, push/pop ≈ 0.87 (Ma 2024)")